整合社会经济数据census文件中的汇总为output_data/merged_census_data.csv


关于犯罪数据，会有一些重复，
合并相同local authority code，汇总重复code的数值：
整理之后为：
output_data/cleaned_crime_total_sum.csv
output_data/cleaned_crime_asb_sum.csv

In [ ]:
import pandas as pd
import glob
import os

# 数据文件夹路径
data_dir = 'census'
# 获取所有 csv 文件路径
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))

# 读取第一个文件，作为主表
main_df = pd.read_csv(csv_files[0])

# 只保留主键和数据列
main_cols = [col for col in main_df.columns if col not in ['date', 'geography']]
main_df = main_df[main_cols]

# 依次合并其它文件
for file in csv_files[1:]:
    df = pd.read_csv(file)
    # 只保留主键和数据列
    cols = [col for col in df.columns if col not in ['date', 'geography']]
    df = df[cols]
    # 合并
    main_df = pd.merge(main_df, df, on='geography code', how='outer')


# 保存合并后的大表格
main_df.to_csv('output_data/merged_census_data.csv', index=False)


In [16]:
df_crime = pd.read_csv('used_data/policeforceareatablesyemarch2025_c5.csv')
print(df_crime.columns.tolist())


['Local Authority code', 'Local Authority name', 'Total recorded crime (excluding fraud)', 'Violence against the person']


删除相同编码，并整合两种犯罪数据

In [4]:
import pandas as pd


df = pd.read_csv('crime/annualsupplementarytablesmarch2025_c16.csv')

# 数值列名（根据你的实际字段调整）
value_cols = [
    'Total recorded ASB',
    'Environmental ASB',
    'Nuisance ASB',
    'Personal ASB'
]

# 先把数值类字段转为数字（去掉逗号，防止报错）
for col in value_cols:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# 按 code 分组，数值列求和，名称取第一个
result_asb = df.groupby('Local Authority code', as_index=False).agg({
    'Local Authority name': 'first',
    'Total recorded ASB': 'sum',
    'Environmental ASB': 'sum',
    'Nuisance ASB': 'sum',
    'Personal ASB': 'sum'
})

print(result_asb)
result_asb.to_csv('output_data/cleaned_crime_asb_sum.csv', index=False)

           Local Authority code  Local Authority name  Total recorded ASB  \
0    Combined Local Authorities                  None             36588.0   
1                     E06000001            Hartlepool              2846.0   
2                     E06000002         Middlesbrough              5548.0   
3                     E06000003  Redcar and Cleveland              3870.0   
4                     E06000004      Stockton-on-Tees              4788.0   
..                          ...                   ...                 ...   
294                   W06000019         Blaenau Gwent              1577.0   
295                   W06000020               Torfaen              1584.0   
296                   W06000021         Monmouthshire              1047.0   
297                   W06000022               Newport              3603.0   
298                   W06000023                 Powys              1268.0   

     Environmental ASB  Nuisance ASB  Personal ASB  
0               2944.0

In [5]:


# 读取文件，自动识别分隔符和去除 BOM
df = pd.read_csv('crime/policeforceareatablesyemarch2025_c5.csv', encoding='utf-8-sig')

# 查看字段名
print(df.columns.tolist())

# 假设字段名如下（请根据实际字段调整）
# ['Local Authority code', 'Local Authority name', 'Total recorded crime (excluding fraud)', 'Violence against the person']

# 如果数值字段带引号或逗号，需要处理
value_cols = [
    'Total recorded crime (excluding fraud)',
    'Violence against the person'
    # 这里可以补充其它数值字段
]

# 去除引号和逗号，转为数字
for col in value_cols:
    df[col] = df[col].astype(str).str.replace('"', '').str.replace(',', '').astype(float)

# 分组汇总
result_crime_total = df.groupby('Local Authority code', as_index=False).agg({
    'Local Authority name': 'first',
    'Total recorded crime (excluding fraud)': 'sum',
    'Violence against the person': 'sum'
    # 其它数值字段也这样写
})

print(result_crime_total.head())

result_crime_total.to_csv('output_data/cleaned_crime_total_sum.csv', index=False)


['Local Authority code', 'Local Authority name', 'Total recorded crime (excluding fraud)', 'Violence against the person']
         Local Authority code  Local Authority name  \
0  Combined Local Authorities                  None   
1                   E06000001            Hartlepool   
2                   E06000002         Middlesbrough   
3                   E06000003  Redcar and Cleveland   
4                   E06000004      Stockton-on-Tees   

   Total recorded crime (excluding fraud)  Violence against the person  
0                                196409.0                      78124.0  
1                                 13416.0                       5100.0  
2                                 24709.0                       9348.0  
3                                 15713.0                       5834.0  
4                                 22452.0                       8182.0  
